In [7]:
import unicodedata as ud
from collections import defaultdict


class AlphabetDetector:
    def __init__(self, no_memory=False):
        self.alphabet_letters = defaultdict(dict)
        self.no_memory = no_memory

    def is_in_alphabet(self, uchr, alphabet):
        if self.no_memory:
            return alphabet in ud.name(uchr)
        try:
            return self.alphabet_letters[alphabet][uchr]
        except KeyError:
            return self.alphabet_letters[alphabet].setdefault(
                uchr, alphabet in ud.name(uchr))

    def only_alphabet_chars(self, unistr, alphabet):
        return all(self.is_in_alphabet(uchr, alphabet)
                   for uchr in unistr if uchr.isalpha())

    def detect_alphabet(self, unistr):
        return set(ud.name(char).split(' ')[0]
                   for char in unistr if char.isalpha())

    def is_cyrillic(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'CYRILLIC') else False

    def is_latin(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'LATIN') else False

    def is_arabic(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'ARABIC') else False

In [8]:
ad = AlphabetDetector()


In [10]:
#states = {(1,'word'), (2,'trans'), (3,'pos')}
auto = {}
pos = set()
#poslist = 
state = 0
word = ['word', 'pos']
Vocab = {}

with open("./rubinchik/rubinchik.txt", 'r', encoding='utf-8') as f:
    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            word[0] = line.strip()
            #print(line)
        elif line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print(line)
        elif ad.is_cyrillic(line.strip()) and state == 2: 
            #if line.split('., ')[0].strip(' \\n') in poslist:
            state = 3
            word[1] = line.split('., ')[0].strip(' \\n')
            pos.add(word[1])
            Vocab[word[0]] = word[1]
            state = 0   
                #print(word[1])
        else:
            state = 0

In [12]:
#pos
with open('./rubinchik/rub_wordlist.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Vocab.items(), key=lambda tup: tup[0], reverse=False)
    for word in dicSorted:
        w.write(word[0]+'\t'+str(word[1])+'\n')
        #i+=1 

In [17]:
len(Vocab)

52612

In [37]:
#print(Vocab)

{'آرتل': 'сущ', 'آرتیکل': 'сущ', 'آردی': 'прил', 'آرزو\u200c': 'сущ', 'آردینه': 'прил', 'آرتیستی': 'нар', 'آرزومندی\u200c': 'сущ', 'آرتیست\u200cبازی': 'сущ', 'آردالو': 'сущ', 'آرژان': 'сущ', 'آرد\u200c': 'сущ', 'آرداله\u200c': 'сущ', 'آرتیست': 'сущ', 'آرزومند\u200c': 'прил', 'آرتیشو': 'сущ', 'آذرشست': 'сущ', 'آرده\u200c': 'сущ', 'آردل\u200c': 'сущ', 'آرتزین': 'прил', 'آرخانگلسک': 'сущ', 'آرتمیا': 'сущ', 'آرداب\u200c': 'сущ', 'آردهاله': 'сущ', 'آرتروز': 'сущ', 'آرخالق\u200c': 'сущ', 'آرتریت': 'сущ', 'آردبیز\u200c': 'сущ', 'آردن\u200c': 'сущ', 'آردواز': 'сущ', 'آرژانتین': 'сущ'}


In [ ]:
#ОНВ آزار‌ [āzār]
#ОНВ آزما(ی) [āzmā(y)]
#гл. ОНВ آژن [āžan]
#ОНВ гл. آشکوخ [āškux] и اشکوخ [aškux]
#1) ОНВ гл. آفریدن

In [3]:
#from __future__ import unicode_literals
#import hazm
#normalizer = Normalizer()
#normalizer.normalize('اصلاح نويسه ها و استفاده از نیم‌فاصله پردازش را آسان مي كند')


from __future__ import unicode_literals
import re, codecs
from .utils import default_verbs
from nltk.tokenize.api import TokenizerI


class WordTokenizer(TokenizerI):
	"""
	>>> tokenizer = WordTokenizer()
	>>> tokenizer.tokenize('این جمله (خیلی) پیچیده نیست!!!')
	['این', 'جمله', '(', 'خیلی', ')', 'پیچیده', 'نیست', '!!!']
	"""

	def __init__(self, verbs_file=default_verbs, join_verb_parts=True):
		self._join_verb_parts = join_verb_parts
		self.pattern = re.compile(r'([؟!\?]+|[:\.،؛»\]\)\}"«\[\(\{])')

		if join_verb_parts:
			self.after_verbs = set([
				'ام', 'ای', 'است', 'ایم', 'اید', 'اند', 'بودم', 'بودی', 'بود', 'بودیم', 'بودید', 'بودند', 'باشم', 'باشی', 'باشد', 'باشیم', 'باشید', 'باشند',
				'شده_ام', 'شده_ای', 'شده_است', 'شده_ایم', 'شده_اید', 'شده_اند', 'شده_بودم', 'شده_بودی', 'شده_بود', 'شده_بودیم', 'شده_بودید', 'شده_بودند', 'شده_باشم', 'شده_باشی', 'شده_باشد', 'شده_باشیم', 'شده_باشید', 'شده_باشند',
				'نشده_ام', 'نشده_ای', 'نشده_است', 'نشده_ایم', 'نشده_اید', 'نشده_اند', 'نشده_بودم', 'نشده_بودی', 'نشده_بود', 'نشده_بودیم', 'نشده_بودید', 'نشده_بودند', 'نشده_باشم', 'نشده_باشی', 'نشده_باشد', 'نشده_باشیم', 'نشده_باشید', 'نشده_باشند',
				'شوم', 'شوی', 'شود', 'شویم', 'شوید', 'شوند', 'شدم', 'شدی', 'شد', 'شدیم', 'شدید', 'شدند',
				'نشوم', 'نشوی', 'نشود', 'نشویم', 'نشوید', 'نشوند', 'نشدم', 'نشدی', 'نشد', 'نشدیم', 'نشدید', 'نشدند',
				'می‌شوم', 'می‌شوی', 'می‌شود', 'می‌شویم', 'می‌شوید', 'می‌شوند', 'می‌شدم', 'می‌شدی', 'می‌شد', 'می‌شدیم', 'می‌شدید', 'می‌شدند',
				'نمی‌شوم', 'نمی‌شوی', 'نمی‌شود', 'نمی‌شویم', 'نمی‌شوید', 'نمی‌شوند', 'نمی‌شدم', 'نمی‌شدی', 'نمی‌شد', 'نمی‌شدیم', 'نمی‌شدید', 'نمی‌شدند',
				'خواهم_شد', 'خواهی_شد', 'خواهد_شد', 'خواهیم_شد', 'خواهید_شد', 'خواهند_شد',
				'نخواهم_شد', 'نخواهی_شد', 'نخواهد_شد', 'نخواهیم_شد', 'نخواهید_شد', 'نخواهند_شد',
			])

			self.before_verbs = set([
				'خواهم', 'خواهی', 'خواهد', 'خواهیم', 'خواهید', 'خواهند',
				'نخواهم', 'نخواهی', 'نخواهد', 'نخواهیم', 'نخواهید', 'نخواهند'
			])

			with codecs.open(verbs_file, encoding='utf8') as verbs_file:
				self.verbs = list(reversed([verb.strip() for verb in verbs_file if verb]))
				self.bons = set([verb.split('#')[0] for verb in self.verbs])
				self.verbe = set([bon +'ه' for bon in self.bons] + ['ن'+ bon +'ه' for bon in self.bons])

	def tokenize(self, text):
		text = self.pattern.sub(r' \1 ', text.replace('\n', ' ').replace('\t', ' '))
		tokens = [word for word in text.split(' ') if word]
		if self._join_verb_parts:
			tokens = self.join_verb_parts(tokens)
		return tokens

	def join_verb_parts(self, tokens):
		"""
		>>> tokenizer = WordTokenizer()
		>>> tokenizer.join_verb_parts(['خواهد', 'رفت'])
		['خواهد_رفت']
		>>> tokenizer.join_verb_parts(['رفته', 'است'])
		['رفته_است']
		>>> tokenizer.join_verb_parts(['گفته', 'شده', 'است'])
		['گفته_شده_است']
		>>> tokenizer.join_verb_parts(['گفته', 'خواهد', 'شد'])
		['گفته_خواهد_شد']
		>>> tokenizer.join_verb_parts(['خسته', 'شدید'])
		['خسته', 'شدید']
		"""

		result = ['']
		for token in reversed(tokens):
			if token in self.before_verbs or (result[-1] in self.after_verbs and token in self.verbe):
				result[-1] = token +'_'+ result[-1]
			else:
				result.append(token)
		return list(reversed(result[1:]))

SystemError: Parent module '' not loaded, cannot perform relative import

In [20]:
path = r"C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\poems\\"
punct = {'،', '«', '»', ':', '!', '؟', '؛','...', '.', '(',')'}

Lemmas = {}

with open(path+'corpus_big.txt', 'r', encoding='utf-8') as f:
    for line in f:
        words = line.split()
        for w in words:
            word = w
            pre, post = '',''
            #haypunct = True
            tokens = []

            if len(word) and word[0] in punct: 
                pre = word[0]
                word = word[1:]
                tokens.append(pre)
            if len(word) and word[-1] in punct:
                post = word[-1]
                word = word[:-1]
                tokens.append(post)
            #if len(word) and word[0] not in punct and word[-1] not in punct:
                #haypunct = False
                #tokens.append(word)
            tokens.append(word)
        
            for token in tokens:
                if token in Lemmas:
                    Lemmas[token] += 1
                else:
                    Lemmas[token] = 1
        
        
with open(path+'frequency_punct1.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Lemmas.items(), key=lambda tup: tup[1], reverse=True)
    for word in dicSorted:
        w.write(word[0]+'\t'+str(word[1])+'\n')
        #i+=1 

In [21]:
len(Lemmas)

36894

In [22]:
len(Vocab)

52612

In [ ]:
'''
with open(path+'corpus_big.txt', 'r', encoding='utf-8') as f:
    for line in f:
        words = line.split()
        for w in words:
            word = w
            pre, post = '',''
            haypunct = True
            tokens = []
            while haypunct:
                if len(word) and word[0] in punct: 
                    pre = word[0]
                    word = word[1:]
                    tokens.append(pre)
                if len(word) and word[-1] in punct:
                    post = word[-1]
                    word = word[:-1]
                    tokens.append(post)
                if len(word) and word[0] not in punct and word[-1] not in punct:
                    haypunct = False
                    tokens.append(word)
                    
                    for token in tokens:
                        if token in Lemmas:
                            Lemmas[token] += 1
                        else:
                            Lemmas[token] = 1
'''

In [ ]:
with open(path+'frequency_punct_alpha.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Lemmas.items(), key=lambda tup: tup[0], reverse=False)
    for word in dicSorted:
        w.write(word[0]+'\n')
        #i+=1 

In [32]:
Corpus = set()
with open('BijanKhan_Distinct_Words.txt','r', encoding='utf-8') as corpus:
    for line in corpus:
        Corpus.add(line)

In [33]:
len(Corpus)

76707

In [38]:
with open(path+'corp_rub_corpus_dif.txt', 'w', encoding='utf-8') as f:
    for w in Lemmas:
        if w in Vocab:
            continue 
        if w in Corpus:
            continue
        f.write(w+'\n')

In [49]:
pre = {'ز','ب','و'}
post = {'ست','م','ت','ش'}

with open(path+'corp_rub_corpus_dif_cut.txt', 'w', encoding='utf-8') as f:
    for w in Lemmas:
        if w in Vocab:
            continue 
        if w in Corpus:
            continue
        word = w
        if len(word) and word[0] in pre:
            if len(word) and (word[1:] in Vocab or word[1:] in Corpus): 
                if Vocab[word[1:]] == 'гл.\n':
                    print(word)
                else:
                    f.write(word[0]+'\t'+word[1:]+'\n')
        if len(word) and word[-1] in post:
            if len(word) and (word[:-1] in Vocab or word[:-1] in Corpus):  
                    f.write(word[:-1]+'\t'+word[-1]+'\n')
                

بباشد
بدادن
بدرودن
بنوشتن
بگرفتیم
بگداختن
بگشادن
باختن
ببریدن
برفتن
بگریختن
بدیدن
بگفتن
بنبشتن
بپرداختن


In [ ]:
#27148

In [43]:
Vocab['افتادن']

'гл.\n'